In [ ]:
import pandas as pd
import utils
import numpy
import matplotlib.pyplot as plt

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
TEAM_NUMBER = "Example 1"

# Path of the stimulus images for the different tasks 
STIMULUS_IMAGE_WITH_AOIS_PATH = "../raw-data/"+TEAM_NUMBER+"/stimulus image with aois/"

# This file should be automatically generated from "2. Event Detection"
GAZE_DATA_WITH_AOIS_AND_EVENTS_FILE = "data/gazeDataWithAOIsAndEvents.csv"

# Display size
DISPLAY_SIZE = (1920,1080)

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 1. Fixations plots
#
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read gaze data with events using pandas library
data = pd.read_csv(GAZE_DATA_WITH_AOIS_AND_EVENTS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview data
data.head()

In [ ]:
# Derive a dataframe with fixation data only
# Keep only relevant colomns
fixationData = data[['Respondent','SourceStimuliName','FixID',
       'Fixation X', 'Fixation Y',
       'Fixation Duration']].copy(deep=True)

# Since the same fixation is assigned to multiple gazes, it is necessary to drop duplicates
fixationData = fixationData.drop_duplicates()

# Then, it is necessary to drop rows with no FixID
fixationData = fixationData.dropna(subset=['FixID'])

# Reset the index of the dataframe
fixationData = fixationData.reset_index(drop=True)

In [ ]:
# Preview fixationData
fixationData.head()

In [ ]:
# Plot fixations

# Iteratate over the participants and tasks and plot fixations for each participant and task
for participant in fixationData['Respondent'].unique():
    
    for task in fixationData['SourceStimuliName'].unique():
    
        # filter in the participant and task data
        participantTaskFixationData= fixationData[(fixationData['Respondent']==participant) & (fixationData['SourceStimuliName']==task)].copy(deep=True)

        # Path to the stimulus image
        image_path = STIMULUS_IMAGE_WITH_AOIS_PATH+task+'.png'

        # Initiate a coordinate system with a background image
        fig, ax = utils.initPlotContainer(image_path)

        # set title
        ax.set_title(f'Participant: {participant}, Task: {task}', fontsize=30)

        # plot fixations (with the size of the markers relative to the fixation duration)
        ax.scatter(participantTaskFixationData["Fixation X"], participantTaskFixationData["Fixation Y"], s= participantTaskFixationData["Fixation Duration"], alpha=0.5)

        #show and close
        plt.show(fig)
        plt.close(fig)
    

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 2. Scan-path plots
#
#
#----------------------------------------------------------------------------------------

In [ ]:
# Iteratate over the participants and tasks and plot the scan-path for each participant and task
for participant in fixationData['Respondent'].unique():
    
    for task in fixationData['SourceStimuliName'].unique():    
    
        # filter in the participant and task data data
        participantTaskFixationData= fixationData[(fixationData['Respondent']==participant) & (fixationData['SourceStimuliName']==task)].copy(deep=True)
        participantTaskFixationData = participantTaskFixationData.reset_index(drop=True)

        # Path to the stimulus image
        image_path = STIMULUS_IMAGE_WITH_AOIS_PATH+task+'.png'

        # Initiate a coordinate system with a background image
        fig, ax = utils.initPlotContainer(image_path)

        # set title
        ax.set_title(f'Participant: {participant}, Task: {task}', fontsize=30)

        # get the length of the fixations dataframe
        fixationsDataFrameLength = len(participantTaskFixationData)

        # plot fixations (with the size of the markers relative to the fixation duration)
        ax.scatter(participantTaskFixationData["Fixation X"], participantTaskFixationData["Fixation Y"], s= participantTaskFixationData["Fixation Duration"], c='green', alpha=0.5)

        # iterate through fixationData
        for index, row in participantTaskFixationData.iterrows():
            # assign an incremental number to each fixation      
            ax.annotate(str(index+1), (row['Fixation X'],row['Fixation Y']), alpha=1, horizontalalignment='center', verticalalignment='center', multialignment='center')
            # connect consecutive pairs of fixations with edges  
            if index<fixationsDataFrameLength-1:
                ax.arrow(row['Fixation X'],row['Fixation Y'], participantTaskFixationData.iloc[index+1]['Fixation X']-row['Fixation X'],participantTaskFixationData.iloc[index+1]['Fixation Y']-row['Fixation Y'], fc='grey', ec='grey', alpha=.7, fill=True, shape='full', width=1, head_width=5, head_starts_at_zero=False, overhang=0)

        #show and close
        plt.show(fig)
        plt.close(fig)

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 3. Heatmap plots
#
#
#----------------------------------------------------------------------------------------

In [ ]:
# Iteratate over the participants and tasks and plot the heatmap for each participant and task
for participant in fixationData['Respondent'].unique():
    
    for task in fixationData['SourceStimuliName'].unique():    
    
        # filter in the participant and task data
        participantTaskFixationData= fixationData[(fixationData['Respondent']==participant) & (fixationData['SourceStimuliName']==task)].copy(deep=True)

        # Path to the stimulus image
        image_path = STIMULUS_IMAGE_WITH_AOIS_PATH+task+'.png'

        # Initiate a coordinate system with a background image
        fig, ax = utils.initPlotContainer(image_path)

        # set title
        ax.set_title(f'Participant: {participant}, Task: {task}', fontsize=30)

        # Gaussian settings
        gwh = 200
        gsdwh = gwh/6
        gaus = utils.gaussian(gwh,gsdwh)

        strt = int(gwh/2)
        heatmapsize = int(DISPLAY_SIZE[1] + 2*strt), int(DISPLAY_SIZE[0] + 2*strt)
        heatmap = numpy.zeros(heatmapsize, dtype=float)

        # create heatmap (adapted from PyGazeAnalyser https://github.com/esdalmaijer/PyGazeAnalyser/)
        for index, row in participantTaskFixationData.iterrows():
            # get x and y coordinates of fixations
            # x and y - indices of heatmap array. must be integers
            x = strt + int(row['Fixation X']) - int(gwh/2)
            y = strt + int(row['Fixation Y']) - int(gwh/2)
            # correct Gaussian size if either coordinate falls outside of
            # display boundaries
            if (not 0 < x < DISPLAY_SIZE[0]) or (not 0 < y < DISPLAY_SIZE[1]):
                hadj=[0,gwh];vadj=[0,gwh]
                if 0 > x:
                    hadj[0] = abs(x)
                    x = 0
                elif DISPLAY_SIZE[0] < x:
                    hadj[1] = gwh - int(x-DISPLAY_SIZE[0])
                if 0 > y:
                    vadj[0] = abs(y)
                    y = 0
                elif DISPLAY_SIZE[1] < y:
                    vadj[1] = gwh - int(y-DISPLAY_SIZE[1])
                # add adjusted Gaussian to the current heatmap
                try:
                    heatmap[y:y+vadj[1],x:x+hadj[1]] += gaus[vadj[0]:vadj[1],hadj[0]:hadj[1]] * row['Fixation Duration']
                except:
                    # fixation was probably outside of display
                    pass
            else:				
                # add Gaussian to the current heatmap
                heatmap[y:y+gwh,x:x+gwh] += gaus * row['Fixation Duration']

        # resize heatmap
        heatmap = heatmap[strt:DISPLAY_SIZE[1]+strt,strt:DISPLAY_SIZE[0]+strt]
        # remove zeros
        lowbound = numpy.mean(heatmap[heatmap>0])
        heatmap[heatmap<lowbound] = numpy.nan
        # draw heatmap on top of image
        ax.imshow(heatmap, cmap='jet', alpha=0.5)

        #show and close
        plt.show(fig)
        plt.close(fig)
